In [81]:
import sys
sys.version

'3.10.8 (main, Nov 24 2022, 08:09:04) [Clang 14.0.6 ]'

In [82]:
%matplotlib inline

import matplotlib.pyplot as plt
plt.xkcd()

import pandas as pd
import numpy as np
import scipy as sp
import quandl
import functools
import seaborn as sns
import plotnine as p9
import statsmodels.api as sm
import math

pd.options.mode.chained_assignment = None 

In [83]:
from functions import *

* Filter by sector, not in the automotive, financial or insurance sector over the entire period Jan 2015 through Jan 2022

In [7]:
df5 = pd.read_csv("filter_by_sector_5000.csv")
ticker_list5 = list(df5['ticker'])

df6 = pd.read_csv("filter_by_sector_6000.csv")
ticker_list6 = list(df6['ticker'])

df7 = pd.read_csv("filter_by_sector_7000.csv")
ticker_list7 = list(df7['ticker'])

df8 = pd.read_csv("filter_by_sector_8000.csv")
ticker_list8 = list(df8['ticker'])

In [8]:
ticker_list = ticker_list5
ticker_list.extend(ticker_list6)
ticker_list.extend(ticker_list7)
ticker_list.extend(ticker_list8)

In [9]:
len(ticker_list)

5076

In [10]:
len(list(set(ticker_list)))

5076

In [11]:
df_ticker_sector = pd.DataFrame(ticker_list, columns =['ticker'])
df_ticker_sector.to_csv('filter_by_sector.csv',index=False)

* end-of-day adjusted closing prices are available over the entire period Jan 2015 through Jan 2022

In [13]:
start_date = '2014-06-30'
end_date = '2022-06-30'

df1 = get_adj_close('LLY',start_date,end_date)
df1

,adj_close
date,
2014-06-30,51.746471
2014-07-01,52.021143
2014-07-02,52.187610
2014-07-03,52.487252
2014-07-07,52.112700
...,...
2022-06-24,323.696565
2022-06-27,325.336819
2022-06-28,315.743821


In [20]:
df52 = pd.read_csv("filter_by_adj_close_5000.csv")
ticker_list52 = list(df52['ticker'])

df62 = pd.read_csv("filter_by_adj_close_6000.csv")
ticker_list62 = list(df62['ticker'])

df72 = pd.read_csv("filter_by_adj_close_7000.csv")
ticker_list72 = list(df72['ticker'])

df82 = pd.read_csv("filter_by_adj_close_8000.csv")
ticker_list82 = list(df82['ticker'])

In [21]:
ticker_list = ticker_list52
ticker_list.extend(ticker_list62)
ticker_list.extend(ticker_list72)
ticker_list.extend(ticker_list82)

In [22]:
len(ticker_list)

1975

In [23]:
len(list(set(ticker_list)))

1975

In [24]:
df_ticker_adj_close = pd.DataFrame(ticker_list, columns =['ticker'])
df_ticker_adj_close.to_csv('filter_by_adj_close.csv',index=False)

* debt/market cap ratio is greater4 than 0.1 somewhere in the period Jan 2015 through Jan 2022 (preferably more than fleetingly)

In [25]:
df_zack = generate_zacks_data('LLY',start_date,end_date)
df_price = get_adj_close('LLY',start_date,end_date)

df = df_price.join(df_zack,how='outer')
df['adj_close'] =  df['adj_close'].fillna(method='ffill')

df1 = debt_to_market_cap(df,df_zack)

df['debt_to_market_cap'] = df1['debt_to_market_cap']

In [34]:
df['debt_to_market_cap']

date
2014-06-30         NaN
2014-07-01         NaN
2014-07-02         NaN
2014-07-03         NaN
2014-07-07         NaN
                ...   
2022-06-24    1.529256
2022-06-27    1.521546
2022-06-28    1.567774
2022-06-29    1.541661
2022-06-30    1.535812
Name: debt_to_market_cap, Length: 2024, dtype: float64

In [32]:
df[['debt_to_market_cap']][df[['debt_to_market_cap']]  > 0.1].count()

debt_to_market_cap    2005
dtype: int64

In [35]:
len(ticker_list)

1975

,ticker,debt_ratio_greater_than_0.1
0,A,0
1,AAL,0
2,AAOI,0
3,AAON,0
4,AAP,0
...,...,...
1970,ZG,0
1971,ZNGA,0
1972,ZNH,0
1973,ZTS,0


In [39]:
def filter_by_debt_ratio(df_ticker,start_date,end_date):

    df_ticker['debt_ratio_greater_than_0.1'] = 0

    for i in range(len(df_ticker)):

        ticker = df_ticker['ticker'][i]

        df_zack = generate_zacks_data(ticker,start_date,end_date)
        df_price = get_adj_close(ticker,start_date,end_date)

        df = df_price.join(df_zack,how='outer')
        df['adj_close'] =  df['adj_close'].fillna(method='ffill')

        df1 = debt_to_market_cap(df,df_zack)

        count = df1[['debt_to_market_cap']][df1[['debt_to_market_cap']]  > 0.1].count()

        df_ticker['debt_ratio_greater_than_0.1'][i] = count

        # if i==100:
        #     break

    return df_ticker

In [40]:
df_ticker_adj_close = pd.read_csv("filter_by_adj_close.csv")

df_ticker_debt_ratio = filter_by_debt_ratio(df_ticker_adj_close,start_date,end_date)

In [42]:
df_ticker_debt_ratio[0:100]

,ticker,debt_ratio_greater_than_0.1
0,A,1983
1,AAL,983
2,AAOI,1835
3,AAON,86
4,AAP,1988
...,...,...
95,AMGN,1999
96,AMKR,2001
97,AMN,2001
98,AMOT,1992


In [36]:
for ticker in ticker_list[0:100]:

    df_zack = generate_zacks_data(ticker,start_date,end_date)
    df_price = get_adj_close(ticker,start_date,end_date)

    df = df_price.join(df_zack,how='outer')
    df['adj_close'] =  df['adj_close'].fillna(method='ffill')

    df1 = debt_to_market_cap(df,df_zack)

    count = df1[['debt_to_market_cap']][df1[['debt_to_market_cap']]  > 0.1].count()

    print(count)
    

debt_to_market_cap    1983
dtype: int64
debt_to_market_cap    983
dtype: int64
debt_to_market_cap    1835
dtype: int64
debt_to_market_cap    86
dtype: int64
debt_to_market_cap    1988
dtype: int64
debt_to_market_cap    2008
dtype: int64
debt_to_market_cap    0
dtype: int64
debt_to_market_cap    2002
dtype: int64
debt_to_market_cap    0
dtype: int64
debt_to_market_cap    1493
dtype: int64
debt_to_market_cap    1821
dtype: int64
debt_to_market_cap    0
dtype: int64
debt_to_market_cap    2008
dtype: int64
debt_to_market_cap    0
dtype: int64
debt_to_market_cap    1981
dtype: int64
debt_to_market_cap    0
dtype: int64
debt_to_market_cap    1999
dtype: int64
debt_to_market_cap    0
dtype: int64
debt_to_market_cap    2003
dtype: int64
debt_to_market_cap    0
dtype: int64
debt_to_market_cap    2003
dtype: int64
debt_to_market_cap    2002
dtype: int64
debt_to_market_cap    430
dtype: int64
debt_to_market_cap    1998
dtype: int64
debt_to_market_cap    0
dtype: int64
debt_to_market_cap    1997
d

In [62]:
df1 = pd.read_csv("filter_by_debt_ratio_1.csv")
df2 = pd.read_csv("filter_by_debt_ratio_2.csv")
df3 = pd.read_csv("filter_by_debt_ratio_3.csv")
df4 = pd.read_csv("filter_by_debt_ratio_4.csv")

In [63]:
df_filter_by_debt_ratio = pd.concat([df1, df2,df3,df4], axis=0)

In [65]:
df_filter_by_debt_ratio[['debt_ratio_greater_than_0.1']][df_filter_by_debt_ratio[['debt_ratio_greater_than_0.1']]  > 1800].count()

debt_ratio_greater_than_0.1    899
dtype: int64

* has feasible calculation of the ratios specified below over the entire period Jan 2015 through Jan 2022, including for at least one PER END DATE no more than one year old

In [66]:
ticker_list = list(df_filter_by_debt_ratio['ticker'])
len(ticker_list)

1975

In [73]:
df_filter_by_debt_ratio['len_of_quarter_data'] = 0
df_filter_by_debt_ratio

,index,ticker,debt_ratio_greater_than_0.1,len_of_quarter_data
0,0,A,1983,0
1,1,AAL,983,0
2,2,AAOI,1835,0
3,3,AAON,86,0
4,4,AAP,1988,0
...,...,...,...,...
470,1970,ZG,1451,0
471,1971,ZNGA,735,0
472,1972,ZNH,1811,0
473,1973,ZTS,1994,0


In [75]:
df_filter_by_debt_ratio = df_filter_by_debt_ratio.set_index('index')
df_filter_by_debt_ratio

,ticker,debt_ratio_greater_than_0.1,len_of_quarter_data
index,,,
0,A,1983,0
1,AAL,983,0
2,AAOI,1835,0
3,AAON,86,0
4,AAP,1988,0
...,...,...,...
1970,ZG,1451,0
1971,ZNGA,735,0
1972,ZNH,1811,0


In [80]:
start_date = '2014-06-30'
end_date = '2022-06-30'

for i in range(len(df_filter_by_debt_ratio)):

    ticker = df_filter_by_debt_ratio.ticker[i]
    df2 = generate_zacks_data(ticker,start_date,end_date)

    df_filter_by_debt_ratio.len_of_quarter_data[i] = len(df2.dropna())
    

KeyboardInterrupt: 

In [78]:
start_date = '2014-06-30'
end_date = '2022-06-30'

df2_list = []

for ticker in ticker_list[0:100]:

    # df2 = generate_zacks_data(ticker,'2015-01-01','2022-02-01')
    df2 = generate_zacks_data(ticker,start_date,end_date)
    df2_list.append(len(df2.dropna()))

In [79]:
df2_list.count(33)

24

In [59]:
len(generate_zacks_data('EBAY',start_date,end_date))

33

In [116]:
df_filtered = pd.read_csv("filtered1.csv")
df_filtered

,ticker,debt_ratio_greater_than_0.1,len_of_quarter_data
0,A,1983,0
1,AAL,983,33
2,AAOI,1835,33
3,AAON,86,1
4,AAP,1988,9
...,...,...,...
1970,ZG,1451,12
1971,ZNGA,735,0
1972,ZNH,1811,1
1973,ZTS,1994,15


In [117]:
df_filter_by_quarter_data = df_filtered.loc[df_filtered['len_of_quarter_data'] >= 30]
df_filter_by_quarter_data = df_filter_by_quarter_data.reset_index(drop=True)

In [118]:
df_filter_by_quarter_data

,ticker,debt_ratio_greater_than_0.1,len_of_quarter_data
0,AAL,983,33
1,AAOI,1835,33
2,AAWW,2002,33
3,ABC,1821,32
4,ABT,1999,31
...,...,...,...
626,XRX,2001,33
627,YORW,1998,33
628,YUM,502,33
629,ZBH,1997,33


In [119]:
list(df_filter_by_quarter_data.ticker)

['AAL',
 'AAOI',
 'AAWW',
 'ABC',
 'ABT',
 'ACCO',
 'ACHC',
 'ACIW',
 'ACM',
 'ACU',
 'ADM',
 'ADMA',
 'ADP',
 'AEP',
 'AES',
 'AGCO',
 'AGYS',
 'AIMC',
 'AIN',
 'AIRT',
 'AIT',
 'AL',
 'ALB',
 'ALCO',
 'ALE',
 'ALG',
 'ALGT',
 'ALIM',
 'ALK',
 'ALKS',
 'ALR',
 'AMC',
 'AMCX',
 'AMED',
 'AMGN',
 'AMKR',
 'AMOT',
 'AMPH',
 'AMRC',
 'AMRS',
 'AMTX',
 'AMZN',
 'AN',
 'ANDE',
 'ANGI',
 'AOS',
 'AOSL',
 'APA',
 'APD',
 'APH',
 'ARC',
 'ARCB',
 'ARLP',
 'ARMK',
 'ARTNA',
 'ARW',
 'ASTE',
 'ATEC',
 'ATI',
 'ATNI',
 'ATR',
 'ATRC',
 'ATRO',
 'ATSG',
 'AVA',
 'AVD',
 'AVT',
 'AVY',
 'AWI',
 'AWK',
 'AWR',
 'AWX',
 'B',
 'BA',
 'BAH',
 'BBGI',
 'BC',
 'BCO',
 'BCOR',
 'BCPC',
 'BDL',
 'BELFB',
 'BERY',
 'BFAM',
 'BG',
 'BGFV',
 'BGS',
 'BHE',
 'BIO',
 'IBM',
 'IBP',
 'IESC',
 'III',
 'IMKTA',
 'INFU',
 'INGR',
 'INOD',
 'INT',
 'IP',
 'IPGP',
 'IT',
 'ITGR',
 'ITRI',
 'JACK',
 'JAZZ',
 'JBLU',
 'JBSS',
 'JBT',
 'JCI',
 'JJSF',
 'JNJ',
 'K',
 'KAI',
 'KALU',
 'KAR',
 'KBAL',
 'KMB',
 'KMI',
 'KNX

In [120]:
df_filter_by_quarter_data[['debt_ratio_greater_than_0.1']][df_filter_by_quarter_data[['debt_ratio_greater_than_0.1']]  > 1990].count()

debt_ratio_greater_than_0.1    420
dtype: int64

In [121]:
df_420 = df_filter_by_quarter_data.loc[df_filter_by_quarter_data['debt_ratio_greater_than_0.1'] > 1990]
df_420 = df_420[['ticker']]
df_420.to_csv("filtered_ticker_420.csv",index=False)
df_420

,ticker
2,AAWW
4,ABT
5,ACCO
6,ACHC
7,ACIW
...,...
623,X
625,XRAY
626,XRX
627,YORW


In [124]:
df_filter_by_quarter_data[['debt_ratio_greater_than_0.1']][df_filter_by_quarter_data[['debt_ratio_greater_than_0.1']]  > 1995].count()

debt_ratio_greater_than_0.1    375
dtype: int64

In [125]:
df_375 = df_filter_by_quarter_data.loc[df_filter_by_quarter_data['debt_ratio_greater_than_0.1'] > 1995]
df_375 = df_375[['ticker']]
df_375.to_csv("filtered_ticker_375.csv",index=False)
df_375

,ticker
2,AAWW
4,ABT
5,ACCO
6,ACHC
7,ACIW
...,...
623,X
625,XRAY
626,XRX
627,YORW


In [127]:
df_filter_by_quarter_data[['debt_ratio_greater_than_0.1']][df_filter_by_quarter_data[['debt_ratio_greater_than_0.1']]  > 1996].count()

debt_ratio_greater_than_0.1    325
dtype: int64

In [128]:
df_325 = df_filter_by_quarter_data.loc[df_filter_by_quarter_data['debt_ratio_greater_than_0.1'] > 1996]
df_325 = df_325[['ticker']]
df_325.to_csv("filtered_ticker_325.csv",index=False)
df_325

,ticker
2,AAWW
4,ABT
5,ACCO
6,ACHC
7,ACIW
...,...
623,X
625,XRAY
626,XRX
627,YORW
